In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from PCR_evaluation.drbart_evaluation import *

In [2]:
log_name = 'train'
model_name = 'pcr_2'
n_processes = 20

with open('../transformed_event_logs/PCR_start_end_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

#test_event_log['time:timestamp'] = test_event_log['time:timestamp_complete']
test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)


known_activities = ['Callback timeout', 'Export result', 'Export to EMS', 'Match patient data', 'Receive sample state', 'Send notification', 'Wait for plate validation', 'timeout']

likelihoods_A = None
likelihoods_A_S = None
likelihoods_A_S_AC = None

/tmp/ipykernel_34681/566510734.py:6: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  test_event_log = pickle.load(f)


In [3]:
N = 1000
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

In [4]:
drbart_model_A = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name/',
                     strict_parser=True)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_PCR_A,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                   },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A = evaluator_A.sample_cases(False, True)

  0%|                                      | 0/4927 [00:00<?, ?it/s]

  0%|                           | 1/4927 [00:29<40:28:37, 29.58s/it]

  0%|                           | 2/4927 [00:30<17:25:48, 12.74s/it]

  0%|                           | 3/4927 [00:31<10:03:23,  7.35s/it]

  0%|                            | 5/4927 [00:32<4:40:31,  3.42s/it]

...

 99%|█████████████████████████▋| 4857/4927 [00:30<00:00, 166.56it/s]

 99%|█████████████████████████▋| 4874/4927 [00:30<00:00, 131.15it/s]

 99%|█████████████████████████▊| 4897/4927 [00:30<00:00, 144.68it/s]

100%|█████████████████████████▉| 4924/4927 [00:30<00:00, 173.85it/s]

100%|██████████████████████████| 4927/4927 [00:30<00:00, 159.87it/s]

In [5]:
np.mean([v.ln() for v in likelihoods_A[0].values()])

Decimal('0.1341359136121724966684197915')

In [6]:
np.mean(get_pscores(likelihoods_A))

np.float64(15458.169085520643)

In [7]:
drbart_model_A_S = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_seconds-in-day/',
                     strict_parser=True)
evaluator_A_S = conduct_evaluation.ConductEvaluation(drbart_model_A_S, SampleOutcomes_DRBART_PCR_A_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                   },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_S = evaluator_A_S.sample_cases(False, True)

  0%|                                      | 0/4927 [00:00<?, ?it/s]

  0%|                           | 1/4927 [00:33<45:26:57, 33.22s/it]

  0%|                           | 2/4927 [00:35<20:14:54, 14.80s/it]

  0%|                           | 3/4927 [00:36<11:41:48,  8.55s/it]

  0%|                            | 4/4927 [00:37<7:39:10,  5.60s/it]

...

 99%|█████████████████████████▌| 4855/4927 [00:25<00:00, 186.99it/s]

 99%|█████████████████████████▊| 4880/4927 [00:25<00:00, 202.86it/s]

 99%|█████████████████████████▊| 4901/4927 [00:25<00:00, 197.80it/s]

100%|█████████████████████████▉| 4922/4927 [00:25<00:00, 199.54it/s]

100%|██████████████████████████| 4927/4927 [00:25<00:00, 190.32it/s]

In [8]:
np.mean([v.ln() for v in likelihoods_A_S[0].values()])

Decimal('0.6132400162074952272115137633')

In [9]:
np.mean(get_pscores(likelihoods_A_S))

np.float64(12492.01480850062)

In [10]:
drbart_model_A_S_D = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_seconds-in-day_day-of-week/',
                     strict_parser=True)
evaluator_A_S_D = conduct_evaluation.ConductEvaluation(drbart_model_A_S_D, SampleOutcomes_DRBART_PCR_A_S_D,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'day_of_week' : 'day_of_week',
                                                   },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_S_D = evaluator_A_S_D.sample_cases(False, True)

  0%|                                      | 0/4927 [00:00<?, ?it/s]

  0%|                           | 1/4927 [00:30<42:07:59, 30.79s/it]

  0%|                           | 2/4927 [00:32<18:23:21, 13.44s/it]

  0%|                           | 3/4927 [00:35<12:01:34,  8.79s/it]

  0%|                            | 6/4927 [00:37<4:47:07,  3.50s/it]

...

 99%|█████████████████████████▋| 4864/4927 [00:25<00:00, 191.81it/s]

 99%|█████████████████████████▊| 4884/4927 [00:25<00:00, 192.88it/s]

100%|█████████████████████████▉| 4904/4927 [00:25<00:00, 193.20it/s]

100%|█████████████████████████▉| 4924/4927 [00:25<00:00, 193.34it/s]

100%|██████████████████████████| 4927/4927 [00:25<00:00, 190.74it/s]

In [11]:
np.mean([v.ln() for v in likelihoods_A_S_D[0].values()])

Decimal('-2.469138632925833650989622111')

In [12]:
np.mean(get_pscores(likelihoods_A_S_D))

np.float64(10341.623527068277)

In [13]:
drbart_model_A_S_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_seconds-in-day_activity-count/',
                     strict_parser=True)
evaluator_A_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_S_AC, SampleOutcomes_DRBART_PCR_A_S_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'known_activities' : known_activities,
                                                   },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_S_AC = evaluator_A_S_AC.sample_cases(False, True)

  0%|                                      | 0/4927 [00:00<?, ?it/s]

  0%|                           | 1/4927 [00:20<28:15:14, 20.65s/it]

  0%|                           | 2/4927 [00:21<12:21:06,  9.03s/it]

  0%|                            | 3/4927 [00:22<7:15:34,  5.31s/it]

  0%|                            | 5/4927 [00:24<3:53:27,  2.85s/it]

...

 98%|███████████████████████████████████████████████████████████████████████████████████████▌ | 4850/4927 [00:25<00:00, 204.31it/s]

 99%|███████████████████████████████████████████████████████████████████████████████████████▉ | 4871/4927 [00:25<00:00, 200.42it/s]

 99%|████████████████████████████████████████████████████████████████████████████████████████▎| 4892/4927 [00:25<00:00, 202.87it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████▋| 4913/4927 [00:25<00:00, 192.94it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████| 4927/4927 [00:25<00:00, 191.35it/s]

In [14]:
cases = test_event_log['case:concept:name'].unique()
case_data = dict([(case_name, evaluator_A_S_AC.get_case_data(case_name)) for case_name in cases])

In [15]:
np.mean([v.ln() for v in likelihoods_A_S_AC[0].values()])

Decimal('0.5393072701819217978830546574')

In [16]:
np.mean(get_pscores(likelihoods_A_S_AC))

np.float64(14200.084059469775)

In [17]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_A_S' : likelihoods_A_S,
    'drbart_model_A_S_D' : likelihoods_A_S_D,
    'drbart_model_A_S_AC' : likelihoods_A_S_AC
}
with open('./pcr_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)